In [12]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [13]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [14]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [15]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [16]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [195]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


In [71]:
# create a feature for number of words in sentence
word_counts.shape

(5318, 3065)

In [202]:
# Find the number of punctuation marks in each sentences
def number_of_punctuation(sentences):
    punct = []
    
    for i in range(len(sentences)):
        temp = []                          # temporary array to hold the tokens
        for token in sentences[0][i]:      # list the type of speech for each word in sentence
            temp.append(token.pos_)        

        temp = pd.DataFrame(temp)
        punct.append(len(temp[temp[0] == 'PUNCT']))    # number of punctuation
    
    return np.array(punct)

# create a feature for number of punctuation in a sentence
num_punct = number_of_punctuation(sentences)
word_counts['Num_Punct'] = num_punct

In [196]:
# Find the number of verb in each sentences
def number_of_verb(sentences):
    verbs = []
    
    for i in range(len(sentences)):
        temp = []                          # temporary array to hold the tokens
        for token in sentences[0][i]:      # list the type of speech for each word in sentence
            temp.append(token.pos_)        

        temp = pd.DataFrame(temp)
        verbs.append(len(temp[temp[0] == 'VERB']))    # number of verbs
    
    return np.array(verbs)

# create a feature for number of punctuation in a sentence
num_verbs = number_of_verb(sentences)
word_counts['Num_Verbs'] = num_verbs

In [197]:
# Find the number of nouns in each sentences
def number_of_noun(sentences):
    nouns = []
    
    for i in range(len(sentences)):
        temp = []                          # temporary array to hold the tokens
        for token in sentences[0][i]:      # list the type of speech for each word in sentence
            temp.append(token.pos_)        

        temp = pd.DataFrame(temp)
        nouns.append(len(temp[temp[0] == 'NOUN']))    # number of nouns
    
    return np.array(nouns)

# create a feature for number of punctuation in a sentence
num_nouns = number_of_noun(sentences)
word_counts['Num_Noun'] = num_nouns

In [198]:
# Find the number of adverbs in each sentences
def number_of_adverb(sentences):
    adverbs = []
    
    for i in range(len(sentences)):
        temp = []                          # temporary array to hold the tokens
        for token in sentences[0][i]:      # list the type of speech for each word in sentence
            temp.append(token.pos_)        

        temp = pd.DataFrame(temp)
        adverbs.append(len(temp[temp[0] == 'ADV']))    # number of punctuation
    
    return np.array(adverbs)

# create a feature for number of punctuation in a sentence
num_adv = number_of_adverb(sentences)
word_counts['Num_Adv'] = num_adv

In [199]:
# Find the number of adjectives in each sentences
def number_of_adjective(sentences):
    adjectives = []
    
    for i in range(len(sentences)):
        temp = []                          # temporary array to hold the tokens
        for token in sentences[0][i]:      # list the type of speech for each word in sentence
            temp.append(token.pos_)        

        temp = pd.DataFrame(temp)
        adjectives.append(len(temp[temp[0] == 'ADJ']))    # number of punctuation
    
    return np.array(adjectives)

# create a feature for number of punctuation in a sentence
num_adj = number_of_punctuation(sentences)
word_counts['Num_Adj'] = num_adj

In [201]:
# Number of words in a sentence
def words_in_sentence(sentences):
    words = []
    
    for i in range(len(sentences)):
        temp = []
        for token in sentences[0][i]:
            if not token.is_punct:
                temp.append(token)
        words.append(len(temp))
        
    return np.array(words)

# create a feature for number of words in a sentence
num_words = words_in_sentence(sentences)
word_counts['Num_Words'] = num_words

In [203]:
word_counts.head(3)

,canary,dinah'll,wriggle,intimate,finally,bark,relative,young,rapid,chamber,...,narrow,skirt,text_sentence,text_source,Num_Verbs,Num_Noun,Num_Adv,Num_Adj,Num_Words,Num_Punct
0,0,0,0,0,0,0,0,0,0,0,...,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,13,12,3,10,57,10
1,0,0,0,0,0,0,0,0,0,0,...,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,11,8,7,7,56,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,5,2,6,4,29,4


## Set target variable and training/test set

In [204]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

## Random Forest

In [176]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.988401253918

Test set score: 0.887687969925


## Logistic Regression

In [206]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3068) (3190,)
Training set score: 0.962695924765

Test set score: 0.921522556391


In [209]:
from sklearn.model_selection import cross_val_score
cross_val_score(lr, X, Y, cv = 5)

array([ 0.8862782 ,  0.8712406 ,  0.92857143,  0.90413534,  0.90207156])

## Gradient Boost

In [178]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.892476489028

Test set score: 0.873120300752


## Same Model, new inputs

In [179]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [180]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [181]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [182]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [210]:
word_counts.head(1)

,canary,dinah'll,wriggle,intimate,finally,bark,relative,young,rapid,chamber,...,narrow,skirt,text_sentence,text_source,Num_Verbs,Num_Noun,Num_Adv,Num_Adj,Num_Words,Num_Punct
0,0,0,0,0,0,0,0,0,0,0,...,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,13,12,3,10,57,10


#### adding new features created

In [211]:
# number of punctuations in new doc
emma_num_punct = number_of_punctuation(emma_sentences)
emma_bow['Num_Punct'] = emma_num_punct

# number of punctuations in new doc
emma_num_verbs = number_of_verb(emma_sentences)
emma_bow['Num_Verbs'] = emma_num_verbs

# number of punctuations in new doc
emma_num_nouns = number_of_noun(emma_sentences)
emma_bow['Num_Noun'] = emma_num_nouns

# number of punctuations in new doc
emma_num_adv = number_of_adverb(emma_sentences)
emma_bow['Num_Adv'] = emma_num_adv

# number of punctuations in new doc
emma_num_adj = number_of_adjective(emma_sentences)
emma_bow['Num_Adj'] = emma_num_adj

# number of words in new doc
emma_num_words = words_in_sentence(emma_sentences)
emma_bow['Num_Words'] = emma_num_words

## Create an orginal set then run the cross validation again

In [214]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.738255033557


col_0,Austen,Carroll
row_0,,
Austen,1660,9
Carroll,693,320


In [215]:
cross_val_score(lr, X_Emma_test, y_Emma_test, cv = 5)

array([ 0.9981378,  0.9981378,  1.       ,  1.       ,  1.       ])

### Compare more models with same input

In [216]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [218]:
# Clean the Emma data.
sense = gutenberg.raw('austen-sense.txt')
sense = re.sub(r'VOLUME \w+', '', sense)
sense = re.sub(r'CHAPTER \w+', '', sense)
sense = text_cleaner(sense)
print(sense[:100])

The family of Dashwood had long been settled in Sussex. Their estate was large, and their residence 


In [219]:
# Parse our cleaned data.
sense_doc = nlp(sense)

In [220]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
sense_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [223]:
# Sense is quite long, let's cut it down to the same length as Alice.
sense_sents = sense_sents[0:len(alice_sents)]

In [224]:
# Build a new Bag of Words data frame for Sense word counts.
# We'll use the same common words from Alice and Persuasion.
sense_sentences = pd.DataFrame(sense_sents)
sense_bow = bow_features(sense_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


#### adding new features created

In [225]:
# number of punctuations in new doc
sense_num_punct = number_of_punctuation(sense_sentences)
sense_bow['Num_Punct'] = sense_num_punct

# number of punctuations in new doc
sense_num_verbs = number_of_verb(sense_sentences)
sense_bow['Num_Verbs'] = sense_num_verbs

# number of punctuations in new doc
sense_num_nouns = number_of_noun(sense_sentences)
sense_bow['Num_Noun'] = sense_num_nouns

# number of punctuations in new doc
sense_num_adv = number_of_adverb(sense_sentences)
sense_bow['Num_Adv'] = sense_num_adv

# number of punctuations in new doc
sense_num_adj = number_of_adjective(sense_sentences)
sense_bow['Num_Adj'] = sense_num_adj

# number of words in new doc
sense_num_words = words_in_sentence(sense_sentences)
sense_bow['Num_Words'] = sense_num_words

In [226]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Sense sentence data with the Alice data from the test set.
X_Sense_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    sense_bow.drop(['text_sentence','text_source'], 1)              # keep the data from Alice data and replace Persuation data
), axis=0)                                                          # with Sense data
y_Sense_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * sense_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Sense_test, y_Sense_test))
lr_Sense_predicted = lr.predict(X_Sense_test)
pd.crosstab(y_Sense_test, lr_Sense_predicted)


Test set score: 0.600298284862


col_0,Austen,Carroll
row_0,,
Austen,1290,379
Carroll,693,320


In [228]:
cross_val_score(lr, X_Sense_test, y_Sense_test, cv = 5)

array([ 0.9981378 ,  0.9981378 ,  0.9981378 ,  1.        ,  0.99813084])